In [ ]:
# Cell 1 — Import libraries & load model/tokenizer

import os
import sys
import re
import numpy as np

from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle

# Ensure access to Member A's preprocessing functions
sys.path.append(os.path.abspath("src"))

from data_preparation import prepare_data   # reuse Member A code

MODEL_PATH = "saved_model/sentiment_model.h5"
TOKENIZER_PATH = "saved_model/tokenizer.pkl"
MAX_LEN = 120

model = load_model(MODEL_PATH)

with open(TOKENIZER_PATH, "rb") as f:
    tokenizer = pickle.load(f)

print("Model and tokenizer loaded successfully.")
